In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import json

In [2]:
import dataset

In [3]:
CONFIG = json.load(open('config.json'))

In [4]:
data = dataset.download_dataset()

Into folder: Dataset/Not/Argetina
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND1.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND2.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND3.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND4.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
	JoinBands
Total de data input: 53794
Dimension data input: (20, 20, 4, 3)


In [5]:
normalized_data = dataset.Normalizer.transform(data)

In [6]:
data_x, data_y = dataset.split_data_x_y(normalized_data)

In [7]:
data_x_train, data_x_test = dataset.split_data_train_test(data_x, CONFIG["data"]["train_split_size"])
data_y_train, data_y_test = dataset.split_data_train_test(data_y, CONFIG["data"]["train_split_size"])

In [8]:
data_x_train.shape

(43035, 20, 20, 4, 3)

In [10]:
dataset_train = dataset.TimeSeriesDataset(data_x_train, data_y_train)
dataset_test = dataset.TimeSeriesDataset(data_x_test, data_y_test)

In [11]:
train_dataloader = DataLoader(dataset_train, CONFIG["training"]["batch_size"], shuffle=True)
test_dataloader = DataLoader(dataset_test, CONFIG["training"]["batch_size"], shuffle=True)

In [ ]:
model = algorithm.CNNModel(config_model=CONFIG["model"], config_training=CONFIG["training"])

In [ ]:
# Entrenamiento del modelo
for epoch in range(CONFIG["training"]["num_epoch"]):
    loss_train, lr_train = model.run_epoch(train_dataloader, is_training=True)
    loss_test, lr_test = model.run_epoch(test_dataloader)
    model.scheduler.step()
    if epoch == 0 or ((epoch+1)%10 == 0):
        print('Epoch[{}/{}] | loss train:{:.6f}, test:{:.6f} | lr:{:.6f}'
              .format(epoch+1, CONFIG["training"]["num_epoch"], loss_train, loss_test, lr_train))

In [ ]:
import torch

correct = 0
total = 0
with torch.no_grad():
    for idx, (x, y) in enumerate(test_dataloader):
        x = x.to(CONFIG["training"]["device"])
        out_batch = model(x)
        _, predicted = torch.max(out_batch, 1)
        
        total += y.size(0)
        correct += (predicted == y).sum().item()

print('Accuracy of the network on the {0} test images: {1} %'.format(total, 100*correct/total))